#  Projeto Diamond - Sistema de recomendação

## Preparando os dados

In [1]:
# importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from scipy.spatial.distance import cdist

In [2]:
# importando dados a serem avaliados
rick_dia = pd.read_csv('data/rick_diamonds.csv')

rick_dia.describe()

plt.hist(rick_dia['z']);

In [3]:
# importando os dados de treinamento do algoritmo
hist_dia = pd.read_csv('data/diamonds.csv')

In [4]:
# Transformando strings da coluna 'clarity' em inteiros discretos
clarity_order = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}

hist_dia['clar'] = hist_dia['clarity'].apply(lambda x: clarity_order[x])
rick_dia['clar'] = rick_dia['clarity'].apply(lambda x: clarity_order[x])

In [5]:
# Transformando strings da coluna 'color' em inteiros discretos
color_order = {'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6}

hist_dia['color_class'] = hist_dia['color'].apply(lambda x: color_order[x])
rick_dia['color_class'] = rick_dia['color'].apply(lambda x: color_order[x])

In [6]:
# Transformando strings da coluna 'cut' em inteiros discretos
cut_order = {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4}

hist_dia['cut_class'] = hist_dia['cut'].apply(lambda x: cut_order[x])
rick_dia['cut_class'] = rick_dia['cut'].apply(lambda x: cut_order[x])

## Primeiro sistema de recomendação

head: 1

Erro: 1019.28957747541

def find_sim(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual
        ao preco da linha do dataset de treinamento mais proxima, de acordo com a distancia
        euclidiana.
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut'], axis=1),df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(1).index, 'price'].mean()
    
    return recommendation

recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim(rick_dia[i:i+1].drop(['clarity', 'color', 'cut'], axis=1)))

rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Testando mais heads

head: 5

Erro: 861.832306397248

def find_sim_5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana.
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut'], axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut'], axis=1)))

rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Utilizando menos colunas
Retirando colunas 'depth' e 'table' do algoritmo

head: 5

Erro: 613.8702581735004

In [ ]:
hist_dia.head()

In [ ]:
def find_sim_col5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table'], axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

In [ ]:
recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_col5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)))

In [ ]:
rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Usando métrica de Hamming
head: 5

Erro: 1546.7734066695095

def find_sim_ham5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        de Hamming. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table'], axis=1), df, metric='hamming')

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_ham5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)))

rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Usando métrica de Manhattan
head: 5

Erro: 1447.6820124115654

In [ ]:
def find_sim_man5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        de Manhattan. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table'], axis=1), df, metric='cityblock')

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

In [ ]:
recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_man5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)))

In [ ]:
rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Cortando colunas categóricas
Retirando colunas 'depth', 'table', 'cut', 'color', 'clarity' do algoritmo

head: 5

Erro: 1447.6820124115654

hist_dia.head()

def find_sim_num5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table', 'clar', 'color_class', 'cut_class'], axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_num5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table', 'clar', 'color_class', 'cut_class'], axis=1)))

rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

##  Cortando colunas 'x', 'y' e 'z'

head: 5

Erro: 689.2085874465001

In [ ]:
def find_sim_xyz5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table', 'x', 'y', 'z'], axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

In [ ]:
recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_xyz5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table', 'x', 'y', 'z'], axis=1)))

In [ ]:
rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Mudando a função de agregação
Retirando colunas 'depth' e 'table' do algoritmo. Aplicando função máximo.

head: 5

Erro: 966.9392524869388

In [ ]:
def find_sim_max5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table'], axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].max()
    
    return recommendation

In [ ]:
recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_max5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)))

In [ ]:
rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Mudando a função de agregação
Retirando colunas 'depth' e 'table' do algoritmo. Aplicando função mediana.

head: 5

Erro: 643.7355361481918

In [ ]:
def find_sim_med5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table'], axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].median()
    
    return recommendation

In [ ]:
recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_med5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)))

In [ ]:
rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Usando métrica de cosenos
head: 5

Erro: 699.3044647791119

In [ ]:
def find_sim_cos5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        de Manhattan. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table'], axis=1), df, metric='cosine')

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

In [ ]:
recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_cos5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)))

In [ ]:
rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Mudando a função de agregação
Retirando colunas 'depth' e 'table' do algoritmo. Aplicando média ponderada pelo ranking da matriz de similaridade.

head: 5

Erro: 603.2720255650976

In [72]:
def find_sim_pon5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table'], axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    top_sim = similarity_index.sort_values(by=0, ascending=False).head(5)
    
    recommendation = sum([hist_dia.loc[i, 'price'] * (top_sim.loc[i,0]/(top_sim.sum())) for i in top_sim.index])

    return recommendation[0]

In [75]:
recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_pon5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)))

100%|██████████| 5000/5000 [01:49<00:00, 45.64it/s]


In [76]:
rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Aumentando as linhas do data set
Retirando colunas 'depth' e 'table' do algoritmo. Após ums previsão, a linha com a previsão é inserida no dataset de controle

head: 5

Erro: 616.7897113043206

In [136]:
def find_sim_lin5(contr, df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''

    df= df.to_frame().T
    distance_vector = cdist(contr.drop('price', axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=contr.index)
    recommendation = contr.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    df['price'] = recommendation
    contr = pd.concat([contr, df], ignore_index=True)

    return recommendation, contr

In [137]:
controle = hist_dia.drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)
recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    rick_dia.loc[i, 'price_predicted'], controle = find_sim_lin5(controle, rick_dia.loc[i, ['carat', 'x', 'y', 'z','clar', 'color_class', 'cut_class']])

100%|██████████| 5000/5000 [04:39<00:00, 17.88it/s]


In [138]:
rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

##  Salvando arquivo

In [139]:
# salvando dados previstos em arquivo .csv 
rick_dia[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'price_predicted']].to_csv('data/rick_dia_recomendado.csv', index=False)